In [1]:
# importando bibliotecas
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [2]:
def read_files(path, name_file, year_date, type_file): 
    _file = f'{path}{name_file}{year_date}.{type_file}'
    colspecs = [(2,10),(10,12),(12,24),(27,39),(56,69),(69,82),(82,95),(108,121),(152,170),(170,188)]

    names = ['data_pregao', 'codbdi', 'sigla_acao', 'nome_acao', 'preco_abertura', 'preco_maximo', 'preco_minimo', 'preco_fechamento', 'qtd_negocios', 'volume_negocios']
    df = pd.read_fwf(_file, colspecs=colspecs, names=names, skiprows=1)
    
    # caso queira visualizar o dataframe, utilizar o código abaixo, comentar a função, var e retornar o DF
    #df = pd.read_fwf('H:\\Meu Drive\\BI\\Python e Outros\\MBA\\PYTHON\\Tratamento_Dados\\Série Histórica Ibovespa\\COTAHIST_A2024.TXT', colspecs=colspecs, names=names, skiprows=1)

    return df
  

In [3]:

def filter_stocks(df):
    df = df[df['codbdi'] == 2]
    df = df.drop(['codbdi'], axis=1)
    
    return df

In [4]:

def ajuste_data(df):
    df['data_pregao'] = pd.to_datetime(df['data_pregao'], format='%Y%m%d')
    return df

In [5]:

def ajustes_valores(df):
    df['preco_abertura'] = (df['preco_abertura'] / 100).astype(float)
    df['preco_maximo'] = (df['preco_maximo'] / 100).astype(float)
    df['preco_minimo'] = (df['preco_minimo'] / 100).astype(float)
    df['preco_fechamento'] = (df['preco_fechamento'] / 100).astype(float)
    return df

In [6]:
def insert_data_into_mysql(path, name_file, year_date, type_file, db_host, db_user, db_password, db_name, table_name):
    try:
        conn = mysql.connector.connect(host=db_host, user=db_user, password=db_password, database=db_name, connect_timeout=120)
        cursor = conn.cursor()
        
        for y in year_date:
            df = read_files(path, name_file, y, type_file)
            df = filter_stocks(df)
            df = ajuste_data(df)
            df = ajustes_valores(df)
            
            # Convertendo os dados do DataFrame para uma lista de tuplas
            data = df.values.tolist()
            
            # Montando a consulta SQL de inserção
            placeholders = ', '.join(['%s'] * len(df.columns))
            columns = ', '.join(df.columns)
            sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
            
            # Executando a consulta SQL
            cursor.executemany(sql, data)

        conn.commit()
        cursor.close()
        conn.close()
        print("Dados inseridos com sucesso no banco de dados!")
    except Error as e:
        print(f"Erro ao conectar ao MySQL: {e}")


In [7]:

# Exemplo de uso da função
insert_data_into_mysql(
    path='H:\\Meu Drive\\BI\\Python e Outros\\MBA\\PYTHON\\Tratamento_Dados\\Série Histórica Ibovespa\\',
    name_file='COTAHIST_A',
    year_date=[2024,2023],
    type_file='TXT',
    db_host='localhost',
    db_user='root',
    db_password='123',
    db_name='acoes',
    table_name='historico_acoes'
)

Dados inseridos com sucesso no banco de dados!
